In [18]:
inputname='成本'  #可填销量/成本 表示预测7月1日的哪个值

In [19]:
import pandas as pd
from keras.models import Sequential
from keras.layers import GRU, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 加载数据
df = pd.read_excel('第三问预测销量和成本_三月.xlsx',sheet_name=inputname)

# 移除第一列
df = df.iloc[:, 1:]

# 初始化预测结果
predictions = []

# 对每一列进行预测
for i in range(df.shape[1]):
    # 获取列数据
    data = df.iloc[:, i].values
    # 删除data中的0值   ！！！如果是预测销量，我们组认为不需要去零处理，而成本需要去0处理
    data = data[data != 0]    
    # 数据归一化
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data.reshape(-1, 1))

    # 创建数据集
    X, y = [], []
    for i in range(len(data)-1):
        X.append(data[i])
        y.append(data[i+1])
    X, y = np.array(X), np.array(y)

    # 重塑输入为 [samples, timesteps, features]
    X = X.reshape((X.shape[0], 1, X.shape[1]))

    # 定义GRU模型
    model = Sequential()
    model.add(GRU(50, activation='relu', input_shape=(1, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # 训练模型
    model.fit(X, y, epochs=100, verbose=0)

    # 对下一步进行预测
    x_input = np.array(data[-1]).reshape((1, 1, 1))
    yhat = model.predict(x_input, verbose=0)

    # 反归一化预测结果
    yhat = scaler.inverse_transform(yhat)

    # 将预测结果添加到预测列表
    predictions.append(yhat[0,0])

# 将预测结果添加到数据框最后一行
df.loc[df.shape[0]] = predictions

d:\anaconda\envs\pytorch\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
  return f(*args, **kwds)


In [20]:
print(predictions)


[3.3788939, 2.009335, 2.808476, 1.5258039, 3.5871575, 2.2143388, 7.5822635, 2.0840404, 4.089809, 3.5796762, 4.342732, 3.4488583, 2.3007581, 2.1651442, 4.0267887, 2.302763, 8.708827, 2.5567837, 10.343449, 3.1524556, 2.7047715, 14.618685, 2.6491804, 6.9729815, 18.248417, 4.1592755, 8.362962, 11.560867, 3.9990964, 2.765815, 3.2781146, 12.626968, 9.25393, 5.50445, 4.567652, 3.3467607, 3.3508177, 12.020744, 9.514058, 9.24626, 13.234177, 2.8908374, 5.6531887, 3.3145325, 1.42458, 1.859303, 12.605825, 2.5184882, 4.632906]


In [21]:

# 将列表转换为DataFrame
predictions = pd.DataFrame(predictions, columns=['Predictions'])

# 将DataFrame写入Excel文件
predictions.to_excel('output.xlsx', index=False)